In [39]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from system_single import AtomCavitySystem
from system_multiple import MultipleAtomsCavitySystem


# Cavity QED  
### Scientific Computing in Quantum Information Science (QIS)  

---

**Sophia Simon**   

22.08.2025


## Outline

1. **Project Idea**  

2. **Theoretical Model**  

3. **Implementation**  

4. **Results: Single Atom**  

5. **Multiple Atoms**  

6. **Spin Echo & Noise**  

7. **Outlook & Extensions**  


## Project Idea

<div align="center">
  <img src="./cavityWithAtom.png" alt="Cavity Illustration" width="600">
</div>


## Theoretical Model

Hamiltonian for a single atom in a cavity:

$$
H = \hbar \omega_c a^\dagger a + \hbar \omega_0 \sigma_z + \hbar g (a \sigma_+ + a^\dagger \sigma_-)
$$

- $a, a^\dagger$: cavity annihilation/creation operators  
- $\sigma_+, \sigma_-$: atomic raising/lowering operators  
- $g$: coupling strength


Hamiltonian for multiple atoms:

$$
H = \hbar \omega_c a^\dagger a + \sum_{j=1}^N \hbar \omega_0 \sigma_z^{(j)} + \hbar g \sum_{j=1}^N \left( a \, \sigma_+^{(j)} + a^\dagger \, \sigma_-^{(j)} \right)
$$

## Implementation in Python 

```python
class AtomCavitySystem:
    def __init__(self, N, g, w0, gamma=0.1, kappa=0.05, beta=0.02):
        self.N = N
        self.g = g
        ...
        
        # Define operators once
        self.a = qt.tensor(qt.destroy(N), qt.identity(2))
        self.sm = qt.tensor(qt.identity(N), qt.destroy(2))
        ...

    def create_hamiltonian(self):
        H_cavity = self.w0 * self.a.dag() * self.a
        H_atom = self.w0 * self.sm.dag() * self.sm
        H_int = self.g * (self.a.dag() * self.sm + self.a * self.sm.dag())
        return H_cavity + H_atom + H_int

        ...
```


## Simulation Methods

- **Quantum toolbox in Python (QuTiP)**
- **Master equation:**  

$$
\dot{\rho} = -i[H, \rho] + \sum_k \mathcal{D}[c_k]\rho
$$

<div align="center">

<pre style="font-size:20px; background-color:#000000; color:#ffffff; padding:15px; border-radius:10px;">
qt.mesolve(H, psi0, times, c_ops, e_ops, options=qt.Options(store_states=True))
</pre>

</div>





- **Collapse operators:** c_ops
  - Cavity decay 
  ```python
  self.c_ops.append(np.sqrt(self.gamma) * self.a)
  ``` 
  - Atomic decay 
  ```python
  self.c_ops.append(np.sqrt(self.kappa) * self.sm)
  ```  
  - Dephasing
  ```python
  self.c_ops.append(np.sqrt(self.beta) * self.sz)
  ``` 




- **Expectation values:**  e_ops
```python
def create_expectation_values(self, number_atom=False, number_cavity=False, coherence_ge=False, coherence_eg=False):
        self.e_ops = []
        if number_atom:
            self.e_ops.append(self.sm.dag() * self.sm)
        if number_cavity:
            self.e_ops.append(self.a.dag() * self.a)
        if coherence_ge:
            self.e_ops.append(self.coh_ge)
        if coherence_eg:
            self.e_ops.append(self.coh_eg)
        return self.e_ops
```



- **Initial state**
```python
# example: '+'
atom = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
```

## Results of Single Atom - Cavity System


```python

def run_simulation(gamma, kappa, beta):
    system = AtomCavitySystem(N=10, g=(0.1 * 2 * np.pi), w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    H = system.create_hamiltonian()

    psi11 = system.create_initial_state('1', 'g')

    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

```


<div align="center">
  <img src="./rabiOszillations.png" alt="Cavity Illustration" width="600">
</div>

<div align="center">
  <img src="./result2Single.png" alt="Cavity Illustration" width="600">
</div>

### State of the Atom on the Bloch Sphere

```python
...

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |0⟩_cavity ⊗ |+⟩_atom
psi11 = system.create_initial_state('0', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=True)

...
```


<div align="center">
  <img src="./plus_state_dec.gif" alt="Cavity Illustration" width="400">
</div>


### Multiple Atoms coupled to the Cavity


```python

class MultipleAtomsCavitySystem:

    ...

    def atom_operator(self, op, atom_index):
        op_list = [qt.qeye(self.N)] 
        for j in range(self.num_atoms):
            op_list.append(op if j == atom_index else qt.qeye(2))
        return qt.tensor(op_list)

    def create_operators(self):
        self.a = qt.tensor([qt.destroy(self.N)] + [qt.qeye(2) for _ in range(self.num_atoms)])
        self.sm_list = [self.atom_operator(qt.destroy(2), i) for i in range(self.num_atoms)]
        self.sz_list = [sm.dag()*sm - sm*sm.dag() for sm in self.sm_list]

    def create_hamiltonian(self):
        H_cavity = self.w0 * self.a.dag() * self.a
        H_atoms = sum(self.w0 * sm.dag() * sm for sm in self.sm_list)
        H_int = sum(self.g * (self.a.dag() * sm + self.a * sm.dag()) for sm in self.sm_list)
        return H_cavity + H_atoms + H_int

    ...

```


<div align="center">
  <img src="./two_atoms.gif" alt="Cavity Illustration" width="600">
</div>


### Spin Echo 

- Introduce static dephasing noise
- Apply π-pulse at t = t_echo to refocus Bloch vector
- Average over multiple realizations


```python
 def evolve_with_static_noise(delta_phi):
    H_noise = H + 0.5 * delta_phi * qt.tensor(qt.qeye(system.N), qt.sigmaz())

    result1 = qt.mesolve(H_noise, psi0, times1, c_ops, [], options=qt.Options(store_states=True))
    rho_mid = result1.states[-1]
    rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()
    result2 = qt.mesolve(H_noise, rho_mid_pulsed, times2, c_ops, [], options=qt.Options(store_states=True))

    return result1.states + result2.states

    # Run multiple realizations
    all_states = []
    for _ in range(num_realizations):
        delta_phi = np.random.normal(0, sigma)
        states = evolve_with_static_noise(delta_phi)
        all_states.append(states)
```

<div align="center">
  <img src="./resultEcho.png" alt="Cavity Illustration" width="600">
</div>

<div align="center">
  <img src="./plus_state_spin_echo.gif" alt="Cavity Illustration" width="600">
</div>

## Outlook & Extension

- Try simulation limits - number of Atoms
- optimize code
- Explore more phenomena
- Introduce detuning and different frequencies 
